# Exercise 2_4 Digital Image Processing

Amirkabir University of Technology

Dr. Rahmati

by Gholamreza Dar

Spring 2022

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import cv2
import os
from tqdm import tqdm
from numba import jit, njit
sns.set_style("dark")


## Functions

### Helpers

In [ ]:
def rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

def gray(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [ ]:
def disp(img, title=None, s=15):
    plt.figure(figsize=(s,s))
    if title is not None:
        plt.title(title)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
    plt.show()

### Bilateral Filter

In [ ]:
obama = rgb(cv2.imread("inputs/P4/putin.png"))

In [ ]:
def bilateral_filter(image, d, c, e):
    return cv2.bilateralFilter(image, d, c, e)

In [ ]:
@jit
def bilateral_filter_ghd(image, d, sigma_d, sigma_r):
    sigma_d = 2*sigma_d*sigma_d
    sigma_r = 2*sigma_r*sigma_r

    image = image.astype(np.float32)
    result = np.zeros(image.shape, dtype=np.float32)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            numerator = np.array([0, 0, 0], dtype=np.float32)
            denominator = np.array([0, 0, 0], dtype=np.float32)
            for k in range(i-d//2, i+d//2):
                for l in range(j-d//2, j+d//2):
                    if 0 <= k < image.shape[0] and 0 <= l < image.shape[1]:
                        w_ijkl = w(image, i, j, k, l, sigma_d, sigma_r) 
                        numerator += image[k, l] * w_ijkl
                        denominator += w_ijkl
            result[i, j] = numerator / denominator
                        
    return result.astype(np.uint8)

In [ ]:
@jit
def w(image, i, j, k, l, sigma_d, sigma_r):
    return np.exp(-((i-k)**2 + (j-l)**2)/(sigma_d)-((image[i][j] - image[k][l])**2)/(sigma_r))

### Make Poster

In [ ]:
def make_poster(image, mask, colors, gamma=1.0, d=75, c=50, e=10, debug=True):
    # 1.Apply bilateral filter to images
    image_bilateral = bilateral_filter(image, d, c, e)
    
    if debug:
        disp(image_bilateral, "Bilateral filtered image")
    
    # 2.Convert to grayscale
    image_bilateral = gray(image_bilateral)

    # 3.Posterization and gamma correction
    colors_count = len(colors)
    image_bilateral =  np.power(image_bilateral, gamma)
    image_bilateral /= image_bilateral.max()/255
    image_bilateral = image_bilateral//(255/(colors_count-1))*(255//(colors_count-1))

    if debug:
        disp(image_bilateral, "Posterized image")
    
    # 4.Color in the foreground
    poster = np.ones_like(image)
    unique_values = np.unique(image_bilateral)
    if debug:
        print(unique_values)

    for i in range(len(unique_values)):
        poster[image_bilateral == unique_values[i]] = colors[i]

    # 5.Color in the background
    poster[:, :poster.shape[1]//2] = np.where(mask==0, colors[2], poster)[:, :poster.shape[1]//2]
    poster[:, poster.shape[1]//2:] = np.where(mask==0, colors[1], poster)[:, poster.shape[1]//2:]

    return poster
    

In [ ]:
def make_poster_tricolor(image, mask, colors, gamma=1.0, d=75, c=50, e=10, debug=True):
    # 1.Apply bilateral filter to images
    image_bilateral = bilateral_filter(image, d, c, e)
    
    if debug:
        disp(image_bilateral, "Bilateral filtered image")
    
    # 2.Convert to grayscale
    image_bilateral = gray(image_bilateral)

    # 3.Posterization and gamma correction
    colors_count = len(colors)
    image_bilateral =  np.power(image_bilateral, gamma)
    image_bilateral /= image_bilateral.max()/255
    image_bilateral = image_bilateral//(255/(colors_count-1))*(255//(colors_count-1))

    if debug:
        disp(image_bilateral, "Posterized image")
    
    # 4.Color in the foreground
    poster = np.ones_like(image)
    unique_values = np.unique(image_bilateral)
    if debug:
        print(unique_values)

    for i in range(len(unique_values)):
        poster[image_bilateral == unique_values[i]] = colors[i]

    # 5.Color in the background
    poster[:, :poster.shape[1]//5] = np.where(mask==0, colors[3], poster)[:, :poster.shape[1]//5]
    poster[:, poster.shape[1]//5:4*poster.shape[1]//5] = np.where(mask==0, colors[2], poster)[:, poster.shape[1]//5:4*poster.shape[1]//5]
    poster[:, 4*poster.shape[1]//5:] = np.where(mask==0, colors[1], poster)[:, 4*poster.shape[1]//5:]

    return poster
    

### Add Text

In [ ]:
from PIL import Image, ImageFont, ImageDraw 

def add_text(poster, text, color, h=200, who=None):
    # Add a bottom margin
    extended_poster = np.zeros((poster.shape[0]+h, poster.shape[1], 3), dtype=np.uint8)
    extended_poster[:poster.shape[0], :poster.shape[1]] = poster
    extended_poster[poster.shape[0]:, :poster.shape[1]] = [0, 48, 80]

    # middle line for alignment
    # extended_poster[:, poster.shape[1]//2:poster.shape[1]//2+1] = [0, 0, 0]

    img = Image.fromarray(extended_poster)
    draw = ImageDraw.Draw(img)
    if who == "obama":
        font = ImageFont.truetype("georgia.ttf", 100, encoding="unic")
        draw.text((poster.shape[1]//2-120,poster.shape[0]+40), text, tuple(color), font=font)
        return img
    elif who == "zelensky":
        font = ImageFont.truetype("georgia.ttf", 90, encoding="unic")
        draw.text((poster.shape[1]//2-170,poster.shape[0]-3), text, tuple(color), font=font)
        return img
    elif who == "putin":
        font = ImageFont.truetype("georgia.ttf", 100, encoding="unic")
        draw.text((poster.shape[1]//2-160,poster.shape[0]+40), text, tuple(color), font=font)
        return img
    else:
        print("Parameter who is not provided")

## Load Images

In [ ]:
obama = rgb(cv2.imread("inputs/P4/obama.png"))
obama_mask = rgb(cv2.imread("inputs/P4/obama_mask.png"))

zelensky = rgb(cv2.imread("inputs/P4/zelensky.png"))
zelensky2 = rgb(cv2.imread("inputs/P4/zelensky2.png"))
zelensky3 = rgb(cv2.imread("inputs/P4/zelensky3.png"))
zelensky_mask = rgb(cv2.imread("inputs/P4/zelensky_mask.png"))

putin = rgb(cv2.imread("inputs/P4/putin.png"))
putin2 = rgb(cv2.imread("inputs/P4/putin4.png"))
putin_mask = rgb(cv2.imread("inputs/P4/putin_mask.png"))

## process

In [ ]:
# Apply bilateral filter to images
obama_bilateral = bilateral_filter(obama, 10, 50, 30)
obama_bilateral2 = bilateral_filter_ghd(obama, 10, 50, 30)
disp(obama_bilateral, s=10)
disp(obama_bilateral2, s=10)

In [ ]:
## REMOVE THIS AFTER TESTS
d = 20
sigma_d = 30
sigma_r = 40

obama_bilateral_test = bilateral_filter_ghd(obama, d, sigma_d, sigma_r)
disp(obama_bilateral_test,title=f"d={d} | sigma_d={sigma_d}, sigma_r={sigma_r}", s=10)

In [ ]:
obama_bilateral = gray(obama_bilateral2) 
disp(obama_bilateral, s=10)

In [ ]:
obama_org = obama_bilateral.copy()

In [ ]:
obama_bilateral = obama_org.copy()

obama_bilateral =  np.power(obama_bilateral, 0.9)
obama_bilateral /= obama_bilateral.max()/255
obama_bilateral = obama_bilateral//(255/4)*(255//4)
disp(obama_bilateral, s=10)

In [ ]:
np.unique(obama_bilateral)

In [ ]:
colors = [
    np.array([255, 0, 0]),
    np.array([0, 255, 0]), 
    np.array([0, 0, 255]),
    np.array([255, 255, 0]),
    np.array([255, 255, 0])]

obama_poster = np.ones_like(obama)
unique_values = np.unique(obama_bilateral)
# Color in the foreground
for i in range(len(unique_values)):
    obama_poster[obama_bilateral == unique_values[i]] = colors[i]

# Color in the background
obama_poster[:, :obama_poster.shape[1]//2] = np.where(obama_mask==0, colors[1], obama_poster)[:, :obama_poster.shape[1]//2]
obama_poster[:, obama_poster.shape[1]//2:] = np.where(obama_mask==0, colors[3], obama_poster)[:, obama_poster.shape[1]//2:]

disp(obama_poster, s=10)

In [ ]:
colors = [
    np.array([0, 48, 80]),
    np.array([218, 20, 21]),
    np.array([112, 150, 160]), 
    np.array([250, 227, 173]),
    np.array([250, 227, 173])
    ]

obama_poster = np.ones_like(obama)
unique_values = np.unique(obama_bilateral)
# Color in the foreground
for i in range(len(unique_values)):
    obama_poster[obama_bilateral == unique_values[i]] = colors[i]

# Color in the background
obama_poster[:, :obama_poster.shape[1]//2] = np.where(obama_mask==0, colors[2], obama_poster)[:, :obama_poster.shape[1]//2]
obama_poster[:, obama_poster.shape[1]//2:] = np.where(obama_mask==0, colors[1], obama_poster)[:, obama_poster.shape[1]//2:]

disp(obama_poster, s=10)

## End to End

### Obama

In [ ]:
# Obama
colors = [
    np.array([0, 48, 80]),
    np.array([218, 20, 21]),
    np.array([112, 150, 160]), 
    np.array([250, 227, 173]),
    np.array([250, 227, 173]),
    ]
obama_poster = make_poster(obama, obama_mask, colors, gamma=1.2,d=20, c=40, e=20, debug=False)
# cv2.imwrite("P4_result/obama_poster.png", bgr(obama_poster))
disp(obama_poster.copy(), "Obama Poster gamma 1.2", s=10)

### Zelensky

In [ ]:
# zelensky
colors = [
    np.array([0, 48, 80]),
    np.array([0, 69, 177]),
    np.array([217, 182, 63]), 
    np.array([255, 235, 175]),
    np.array([255, 235, 175]),
    ]
zelensky_poster = make_poster(zelensky, zelensky_mask, colors, gamma=0.9,d=20, c=50, e=20, debug=False)
# cv2.imwrite("P4_result/zelensky_poster_v1.png", bgr(zelensky_poster))
disp(zelensky_poster.copy(), "Zelensky Poster", s=6)

In [ ]:
# zelensky
colors = [
    np.array([0, 48, 80]),
    np.array([0, 69, 177]),
    np.array([217, 182, 63]), 
    np.array([255, 235, 175]),
    np.array([255, 235, 175]),
    ]
zelensky_poster = make_poster(zelensky2, zelensky_mask, colors, gamma=0.9,d=20, c=50, e=20, debug=False)
# cv2.imwrite("P4_result/zelensky_poster_v2.png", bgr(zelensky_poster))
disp(zelensky_poster.copy(), "Zelensky Poster", s=6)

In [ ]:
# zelensky
colors = [
    np.array([0, 48, 80]),
    np.array([0, 69, 177]),
    np.array([217, 182, 63]), 
    np.array([255, 235, 175]),
    np.array([255, 235, 175]),
    ]
zelensky_poster = make_poster(zelensky3, zelensky_mask, colors, gamma=0.9,d=20, c=50, e=20, debug=False)
# cv2.imwrite("P4_result/zelensky_poster_v3.png", bgr(zelensky_poster))
disp(zelensky_poster.copy(), "Zelensky Poster", s=6)

### Putin

In [ ]:
# Putin
colors = [
    np.array([0, 48, 80]),
    np.array([206, 42, 29]),
    np.array([0, 55, 161]), 
    np.array([222, 220, 235]),
    np.array([222, 220, 235]),
    ]
putin_poster = make_poster_tricolor(putin, putin_mask, colors, gamma=0.85,d=20, c=40, e=20, debug=False)
# cv2.imwrite("P4_result/putin_poster_v1.png", bgr(putin_poster))
disp(putin_poster.copy(), "Putin Poster", s=6)

In [ ]:
# Putin
colors = [
    np.array([0, 48, 80]),
    np.array([206, 42, 29]),
    np.array([0, 55, 161]), 
    np.array([222, 220, 235]),
    np.array([222, 220, 235]),
    ]
putin_poster = make_poster_tricolor(putin2, putin_mask, colors, gamma=0.85,d=20, c=40, e=40, debug=False)
cv2.imwrite("P4_result/putin_poster_v2.png", bgr(putin_poster))
disp(putin_poster.copy(), "Putin Poster", s=6)

## Add Text

### Obama

In [ ]:
obama_poster_with_text = add_text(obama_poster, "Hope", (112, 150, 160), who="obama")
obama_poster_with_text = np.array(obama_poster_with_text)
# cv2.imwrite("P4_result/obama_poster_with_text3.png", bgr(obama_poster_with_text))
disp(obama_poster_with_text, "Obama Poster with Text", s=6)

### Zelensky

In [ ]:
zelensky_poster_with_text = add_text(zelensky_poster, "  Слава\nУкраїні!", (250, 227, 173), who="zelensky")
zelensky_poster_with_text = np.array(zelensky_poster_with_text)
# cv2.imwrite("P4_result/zelensky_poster_with_text.png", bgr(zelensky_poster_with_text))
disp(zelensky_poster_with_text, "Zelensky Poster with Text", s=6)


### Putin

In [ ]:
putin_poster_with_text = add_text(putin_poster, "Россия", (222, 220, 235), who="putin")
putin_poster_with_text = np.array(putin_poster_with_text)
# cv2.imwrite("P4_result/putin_poster_with_text.png", bgr(putin_poster_with_text))
disp(putin_poster_with_text, "Putin Poster with Text", s=6)
